In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
import torch 
import torch.nn as nn

# Read csv

In [5]:
train_df = pd.read_csv('/Users/sachin/Data/IMDB/train_imdb.csv')

In [6]:
test_df = pd.read_csv('/Users/sachin/Data/IMDB/test_imdb.csv')

# Read Pickles

In [8]:
train_data = None

In [9]:
test_data = None

# Model Architecture

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTM_Model, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.sigmoid = nn.Sigmoid()
        
        self.hidden = None
        self.cell = None

    def forward(self, x):
  
        if self.hidden is None:
            h0 = torch.randn(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

            # Initialize cell state
            c0 = torch.randn(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        else:
            h0 = self.hidden
            c0 = self.cell
            
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        self.hidden = hn
        self.cell = cn
        
        out = self.fc(out[:, -1, :]) 
       
        out = self.sigmoid(out)
        return out

In [ ]:
input_dim = 768
hidden_dim = 256
layer_dim = 1
output_dim = 1
num_epochs = 10

In [ ]:
model = LSTM_Model(input_dim, hidden_dim, layer_dim, output_dim)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [ ]:
batch_size = 32
num_batches = train_df.shape[0]//batch_size

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch : {epoch}")
    for i in range(num_batches+1):
        
        if i == num_batches:
            batch = train_df.shape[0] - i*batch_size
        else:
            batch = batch_size
            
        x1 = np.zeros((batch, 512, 768))
        
        for j in range(batch):
            x1[j] = train_data[i*batch_size + j]
            
        label1 = df['y'][i*batch_size:batch]
        
        # Convert x,y into tensors
        x2 = torch.tensor(x1, dtype=torch.float)
        label2 = torch.tensor(np.array(label1), dtype=torch.float)
        
        # Pass x into the mdoel
        output = model(x2)
        
        # Calculate Loss
        loss = criterion(output, label2)

        # Getting gradients w.r.t. parameters
        loss.backward()
        print("Loss : ", loss.item())
        
        # Updating parameters
        optimizer.step()